<a href="https://colab.research.google.com/github/nadienuncanada/BigData/blob/main/BigData_TP3_SparkStreaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BigData 2024 - Spark
# TP3 - Nicolas Ricciardi, Ignacio Melo

## Inicializacion

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
ruta = '/content/drive/MyDrive/BigData'
GENERALinputPath= ruta + '/input'

# Instalamos Spark para Python
!pip install pyspark

import os

# Instalamos Java SDK 8
!apt-get install -y openjdk-8-jdk -qq > /dev/null
!echo $(/usr/libexec/java_home -v 1.8)

#set environment variable
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!echo 2 | update-alternatives --config java

# Creamos el contexto de Spark
from pyspark import SparkContext
sc = SparkContext("local", "Prueba")

from pyspark.streaming import StreamingContext
ssc = StreamingContext(sc, 10)

In [ ]:
''' #TIRA ERROR:

root_path = GENERALinputPath + "/Libros/l1.txt"
print(root_path)


ssc.checkpoint("buffer")

streamOriginal = ssc.textFileStream(root_path)
stream = streamOriginal.flatMap(lambda t: t.split(" "))
stream = stream.map(lambda t: (t, 1))
stream = stream.reduceByKey(lambda t1, t2: t1+t2)
stream.pprint()

def fUpdate(newValues, history):
	if(history == None):
		history = 0
	if(newValues == None):
		newValues = 0
	else:
		newValues = sum(newValues)
	return newValues + history

history = stream.updateStateByKey(fUpdate)
history.pprint()

ssc.start()
ssc.awaitTermination()'''

#Enunciado:


**Pautas generales**

##Enunciado


Basado en el problema presentado en el TP1 y TP2, implemente una solución en Spark streaming
que permita actualizar la información almacenada por un sistema recomendador de misiones.


En este trabajo solo se pide el script que mantiene actualizado el “conocimiento” del sistema
recomendador. No se pide implementar el módulo que hace la recomendación.


Para mantener el sistema actualizado el servidor genera un stream de tuplas
<id_user, id_mission, tiempo_juego>. Se ofrecen 50 archivos con tuplas de misiones jugadas
para poder simular el stream de datos.


Cada una de las misiones inicia con un score igual a cero. Y la actualización del score de cada
misión jugada se realiza en todas las ventanas temporales. Por lo tanto en cada ventana
temporal deberá:


1) Acumular, para cada misión, el tiempo promedio de juego entre todos los usuarios que
la jugaron. El tiempo promedio será acumulado al score de la misión.


2) Además, en cada ventana temporal procesada el sistema deberá ir descontando del
score de todas las misiones un valor de antigüedad A preestablecido.


Ejemplo 1: si una misión con un score de 30, y A = 0.5, en una ventana temporal no fue jugada
entonces su nuevo score será 30 – 0.5 = 29.5


Ejemplo 2: si una misión con un score de 30, y A = 1, en una ventana temporal fue juagada
durante 10 segundos, entonces su nuevo score será 30 + 10 – 1 = 39.


Notar que la antigüedad se descuenta en todas las ventanas temporales, sin importar si la misión
fue jugada o no.

#Resolucion:

In [ ]:
root_path = GENERALinputPath + "/MisionesStreaming"
print(root_path)
A = 1 # 1 de descuento por cada ventana temporal

ssc.checkpoint("buffer")

streamOriginal = ssc.textFileStream(root_path) #<id_user, id_mission, tiempo_juego>

streamOriginal = streamOriginal.map(lambda line: line.split("\t"))  # Ajusta el separador según corresponda
stream = streamOriginal.map(lambda elem: (elem[1], elem[2]))
stream = stream.map(lambda mision: (mision[0], (int(mision[1]), 1))) #(id, (tiempo, cantidad de tiempos acumulados))

stream = stream.reduceByKey(lambda t1, t2: (t1[0]+t2[0],t1[1]+t2[1]))#acumulamos por id de mision el tiempo_juego total y la cantidad de "partidas acumuladas".
stream = stream.map(lambda mision: (mision[0], mision[1][0]/mision[1][1]))#id_mison,(total,cantidad)

stream.pprint()

def fUpdate(newValues, history):
	if(history == None):
		history = 0
	if(newValues == None):
		newValues = 0
	else:
		newValues = sum(newValues)
	return newValues + history - A

history = stream.updateStateByKey(fUpdate)
history.pprint()

ssc.start()
ssc.awaitTermination()

/content/drive/MyDrive/BigData/input/MisionesStreaming
-------------------------------------------
Time: 2024-12-04 21:37:10
-------------------------------------------

-------------------------------------------
Time: 2024-12-04 21:37:10
-------------------------------------------

-------------------------------------------
Time: 2024-12-04 21:37:20
-------------------------------------------

-------------------------------------------
Time: 2024-12-04 21:37:20
-------------------------------------------

-------------------------------------------
Time: 2024-12-04 21:37:30
-------------------------------------------
('5', 446.0)
('22', 448.0)
('17', 447.0)
('6', 367.0)
('15', 439.3333333333333)
('10', 402.0)
('14', 304.0)
('23', 401.8333333333333)
('13', 457.5)
('7', 356.0)
...

-------------------------------------------
Time: 2024-12-04 21:37:30
-------------------------------------------
('5', 445.0)
('22', 447.0)
('17', 446.0)
('6', 366.0)
('15', 438.3333333333333)
('10', 401.

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 